In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('numpy: {}'.format(numpy.__version__))





python: 3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]
NLTK: 3.6.1
Sklearn: 0.24.1
Pandas: 1.2.4
numpy: 1.19.5


In [2]:
## Loading Dataset

In [3]:
import pandas as pd
import numpy as np

# loading SMS dataset 
data = pd.read_table('SMSSpamCollection', header = None, encoding ='utf-8' )

In [4]:
#gathering information about loaded dataset
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [5]:
# Checking for class distribution
classes = data[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64
